In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, timedelta
import json
import itertools
from tqdm import tqdm

In [2]:
import os
os.chdir("/content/drive/MyDrive/projects/Store_Sales-Time_Series_Forecasting/")

In [3]:
# 特徴量
#df_add_f = pd.read_csv("store-sales-time-series-forecasting/train_add_f.csv", index_col=0, parse_dates=True)
df_add_f = pd.read_csv("store-sales-time-series-forecasting/train_add_f_noCh.csv", index_col=0, parse_dates=True).fillna(0)
df_add_f = df_add_f.loc["2017-04-21":]
df_add_f.head()

,dcoilwtico,avg_oil,dayofweek_cos,dayofweek_sin,wd,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,wageday,school_season
2017-04-21,49.640000,51.190952,-0.222521,-9.749279e-01,1,0,0,0,0,0,0,0,1
2017-04-22,49.393333,51.021429,0.623490,-7.818315e-01,0,0,0,0,0,0,0,0,1
2017-04-23,49.146667,50.851905,1.000000,-2.449294e-16,0,0,0,0,0,0,0,0,1
2017-04-24,48.900000,50.682381,0.623490,7.818315e-01,1,0,0,0,0,0,0,0,1
2017-04-25,49.220000,50.512857,-0.222521,9.749279e-01,1,0,0,0,0,0,0,0,1


In [4]:
# trainデータ
df_train = pd.read_csv("store-sales-time-series-forecasting/train.csv", index_col=1, parse_dates=True)
#df_train = df_train.set_index(['date', 'store_nbr', 'family']).sort_index()

# # 売り上げ0
# c = df_train.groupby(["store_nbr", "family"]).sales.sum().reset_index().sort_values(["family","store_nbr"]).query(" sales==0 ")
# # cに該当するものはtrainから消している
# outer_join = df＿train.reset_index().merge(c, how = 'outer', indicator = True)
# df＿train = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1) 
# df_train = df_train.set_index(['date', 'store_nbr', 'family']).sort_index()
# y = df_train.unstack(['store_nbr', 'family']).loc["2017-04-21":]
# y.head()
df_train.head()

,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0
2013-01-01,1,1,BABY CARE,0.0,0
2013-01-01,2,1,BEAUTY,0.0,0
2013-01-01,3,1,BEVERAGES,0.0,0
2013-01-01,4,1,BOOKS,0.0,0


In [18]:
df_test = pd.read_csv("store-sales-time-series-forecasting/test.csv", index_col=1, parse_dates=True)
df_test.head()

,id,store_nbr,family,onpromotion
date,,,,
2017-08-16,3000888,1,AUTOMOTIVE,0
2017-08-16,3000889,1,BABY CARE,0
2017-08-16,3000890,1,BEAUTY,2
2017-08-16,3000891,1,BEVERAGES,20
2017-08-16,3000892,1,BOOKS,0


In [28]:
conf = json.load(open("config/config_simple_linear.json"))
cutoffs = conf["periods"]
all_params = [
    dict(zip(conf["cv_params"].keys(), v)) for v in itertools.product(*conf["cv_params"].values())
]
conf

{'cv_params': {},
 'params': {'n_jobs': -1},
 'periods': [{'end': '2017-05-31', 'start': '2017-04-21'},
  {'end': '2017-06-30', 'start': '2017-06-01'},
  {'end': '2017-08-15', 'start': '2017-07-01'}],
 'train_start': '2017-04-21',
 'sub-train_start': '2017-04-21',
 'sub-train_end': '2017-08-15',
 'sub_start': '2017-08-16',
 'sub_end': '2017-08-31'}

In [6]:
cutoffs

[{'end': '2017-05-31', 'start': '2017-04-21'},
 {'end': '2017-06-30', 'start': '2017-06-01'},
 {'end': '2017-08-15', 'start': '2017-07-01'}]

In [7]:
all_params

[{}]

In [8]:
def walk_forward_validation(df_X, df_y , model, cutoffs):
  indexes = []
  y_vals = []
  y_preds =[]  
  for i in range(len(cutoffs)-1):
    train_start = cutoffs[i]["start"]
    train_end = cutoffs[i]["end"]
    val_start = cutoffs[i+1]["start"]
    val_end = cutoffs[i+1]["end"]

    X_train = df_X.loc[train_start : train_end]
    y_train = df_y.loc[train_start : train_end]
    X_val = df_X.loc[val_start : val_end]
    y_val = df_y.loc[val_start : val_end] 
    y_vals.extend(y_val)

    # モデルの学習
    model.fit(X_train, y_train)
    # 検証データの予測
    y_preds.extend(model.predict(X_val))
    indexes.extend(df_y.loc[val_start : val_end].index)

  df_cv = pd.DataFrame({"y" : y_vals, "y_pred" : y_preds}, index=indexes)
  return df_cv

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

result = []
sub = []

for family in tqdm(df_train["family"].unique()):
  for nbr in df_train["store_nbr"].unique():
    df_y = df_train.query(" (family==@family) & (store_nbr==@nbr) ").sales
    rmsles = []
    for params in all_params:
      model = LinearRegression(**params)
      df_cv = walk_forward_validation(df_add_f, df_y , model, cutoffs)

      y_true = df_cv["y"]
      y_pred = df_cv["y_pred"]
      y_pred[y_pred<0] = 0 # 予測が0未満の場合は0に置換

      rmsles.append(mean_squared_log_error(y_true, y_pred, squared=False))
    
    best_params = pd.DataFrame({"all_params" : all_params, "rmsles" : rmsles}).sort_values("rmsles").iloc[0].to_list()
    best_params.extend([family, nbr])
    result.append(best_params)

100%|██████████| 33/33 [02:29<00:00,  4.52s/it]


In [50]:
df_result = pd.DataFrame(result)
df_result.columns = ["params", "RMSLE", "family", "store_nbr"]
df_result = df_result.sort_values(["store_nbr", "family"])
df_result.head()

,params,RMSLE,family,store_nbr
0,{},0.635475,AUTOMOTIVE,1
54,{},0.000000,BABY CARE,1
108,{},0.798935,BEAUTY,1
162,{},0.276208,BEVERAGES,1
216,{},0.560853,BOOKS,1


### 一番良かったparamで提出用のデータを予測

In [55]:
familys = df_result["family"].unique().tolist()
store_nbrs = df_result["store_nbr"].unique().tolist()
a = list(itertools.product(store_nbrs, familys))
params = df_result["params"].tolist()

subtrain_start = conf["sub-train_start"]
subtrain_end = conf["sub-train_end"]
sub_start = conf["sub_start"]
sub_end = conf["sub_end"]

sub = []

for (nbr, family), param in tqdm(zip(a, params)):
  X = df_add_f.loc[subtrain_start : subtrain_end]
  y = df_train.loc[subtrain_start : subtrain_end].query(" (family==@family) & (store_nbr==@nbr) ").sales
  model = LinearRegression(**param)
  model.fit(X, y)
  X_sub = df_add_f.loc[sub_start : sub_end]
  y_sub = model.predict(X_sub)
  sub.extend(y_sub)

1782it [00:29, 60.16it/s]


In [58]:
df_sub = pd.read_csv("store-sales-time-series-forecasting/sample_submission.csv")
df_sub["sales"] = sub
df_sub.head()

,id,sales
0,3000888,2.928436
1,3000889,4.523167
2,3000890,6.315121
3,3000891,3.553975
4,3000892,2.425374


In [81]:
model_name = "Linear"
filename = datetime.now().strftime(format="%y%m%d%H%M") + '_' + model_name
df_sub.to_csv(f"submission/{filename}.csv", index=None)